In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [86]:
import numpy as np
import pandas as pd
import random
import os
from scipy.stats import kurtosis, skew

import seaborn as sns
import matplotlib.pyplot as plt

# import hdbscan
os.getpid()

import seaborn as sns
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold, train_test_split
import lightgbm as lgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

# from tqtm import tqtm

In [2]:
# import sys
# !{sys.executable} -m pip install hdbscan

In [3]:
csv_file_folder =  '../../data/input'
file_folder = '../../data/feature'
os.listdir(csv_file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [5]:
df_train2 = pd.read_pickle('../select_features/tmp_df_train', compression='gzip')

In [54]:
train.type.value_counts()

3JHC    1510379
2JHC    1140674
1JHC     709416
3JHH     590611
2JHH     378036
3JHN     166415
2JHN     119253
1JHN      43363
Name: type, dtype: int64

In [7]:
df_train2.type.value_counts()

0    709416
Name: type, dtype: int64

In [55]:
test_molecule_names = train[train['type']=='1JHC'].molecule_name.unique().tolist()

In [58]:
len(test_molecule_names), len(train.molecule_name.unique().tolist()), train[train['type']=='1JHC'].shape

(84747, 85003, (709416, 6))

In [57]:
columns0 = ['tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_angle_6', 'dist_C_4_x', 'cos_f1', 'atom_index_1_n_cycle', 'cos_f0_f1', 'tertiary_atom_4', 'gap', 'dist_O_2_x', 'yukawa_O.y', 'd_8_2', 'atom_index_1_eigv_max', 'tertiary_distance_9', 'range_dist_no_bond_x']

In [121]:
# columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2[columns].values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)

LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'mae',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.2,
    'num_leaves': 256,
    'min_child_samples': 79,
    'max_depth': 9,
    'subsample_freq': 1,
    'subsample': 0.9,
    'bagging_seed': 11,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0
}

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
        verbose=100, early_stopping_rounds=200)

y_pred = model.predict(X_val)
np.log(mean_absolute_error(y_val, y_pred))

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's l1: 0.853667	valid_1's l1: 0.931333
[200]	valid_0's l1: 0.710307	valid_1's l1: 0.820019
[300]	valid_0's l1: 0.623902	valid_1's l1: 0.760104
[400]	valid_0's l1: 0.564545	valid_1's l1: 0.722912
[500]	valid_0's l1: 0.522169	valid_1's l1: 0.697593
[600]	valid_0's l1: 0.487984	valid_1's l1: 0.678786
[700]	valid_0's l1: 0.457387	valid_1's l1: 0.663571
[800]	valid_0's l1: 0.430219	valid_1's l1: 0.650643
[900]	valid_0's l1: 0.407637	valid_1's l1: 0.640668
[1000]	valid_0's l1: 0.387826	valid_1's l1: 0.632219
[1100]	valid_0's l1: 0.368615	valid_1's l1: 0.624253
[1200]	valid_0's l1: 0.352771	valid_1's l1: 0.617983
[1300]	valid_0's l1: 0.33855	valid_1's l1: 0.612801
[1400]	valid_0's l1: 0.323716	valid_1's l1: 0.607404
[1500]	valid_0's l1: 0.310481	valid_1's l1: 0.602823
Did not meet early stopping. Best iteration is:
[1500]	valid_0's l1: 0.310481	valid_1's l1: 0.602823


NameError: name 'mean_absolute_error' is not defined

In [123]:
np.log(mean_absolute_error(y_val, y_pred))

-0.5061319030606779

In [10]:
structures = pd.read_csv(f'{csv_file_folder}/structures.csv')
structures = structures[structures['molecule_name'].isin(test_molecule_names)]

In [11]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [12]:
df_edge= pd.read_pickle(f'{file_folder}/df_edge', compression='gzip') 
df_edge = df_edge[df_edge['molecule_name'].isin(test_molecule_names)]
df_edge = df_edge[['molecule_name', 'atom_index_0', 'atom_index_1','distance','angle','angle2center']]

In [13]:
df_edge = pd.merge(df_edge, structures[['molecule_name','atom_index','atom']], how='left', left_on=['molecule_name','atom_index_0'], right_on=['molecule_name', 'atom_index'])
df_edge = df_edge.drop(columns=['atom_index'])
df_edge = df_edge.rename(columns={'atom':'atom_0'})

In [14]:
df_edge = pd.merge(df_edge, structures[['molecule_name','atom_index','atom']], how='left', left_on=['molecule_name','atom_index_1'], right_on=['molecule_name', 'atom_index'])
df_edge = df_edge.drop(columns=['atom_index'])
df_edge = df_edge.rename(columns={'atom':'atom_1'})

In [15]:
df_edge.head()

,molecule_name,atom_index_0,atom_index_1,distance,angle,angle2center,atom_0,atom_1
0,dsgdb9nsd_056913,0.0,1.0,1.515609,0.813115,0.825041,C,C
1,dsgdb9nsd_056913,0.0,2.0,2.545306,1.964539,1.677354,C,C
2,dsgdb9nsd_056913,0.0,3.0,2.531450,2.019837,1.957795,C,N
3,dsgdb9nsd_056913,0.0,4.0,2.626634,2.093707,2.281331,C,C
4,dsgdb9nsd_056913,0.0,5.0,3.135969,1.608493,1.792268,C,C


In [92]:
structures.atom.unique()

array(['C', 'H', 'N', 'O', 'F'], dtype=object)

In [ ]:
list_ = []
idx = 0
N_ = len(df_edge.groupby(by=['molecule_name', 'atom_index_0']))
for c,g in df_edge.groupby(by=['molecule_name', 'atom_index_0']):
    
    d_ = {}
    d_['molecule_name'] = c[0]
    d_['atom_index'] = int(c[1])
    
    for a in ['C', 'H', 'N', 'O', 'F']:
        
        ary = g[g['atom_1']==a]['distance'].values
        
        d_[f'w_{a}_max'] = np.max(ary) if ary.shape[0]>0 else 0
        d_[f'w_{a}_min'] = np.min(ary) if ary.shape[0]>0 else 0
        d_[f'w_{a}_mean'] = np.mean(ary) if ary.shape[0]>0 else 0
        d_[f'w_{a}_std'] = np.std(ary) if ary.shape[0]>0 else 0
        d_[f'w_{a}_kurtosis'] = kurtosis(ary) if ary.shape[0]>0 else 0
        d_[f'w_{a}_skew'] = skew(ary) if ary.shape[0]>0 else 0
    
    
#     atom_index_list = list(np.arange(g.shape[0]+1))
#     atom_index_list.remove(int(c[1]))
#     distances = g.distance.values
#     for i in range(10):
#         d_[f'd_to_{i}'] = distances[i] if i < len(distances)-1 else 0
        
#     d_['min'] = np.min(distances)
#     d_['max'] = np.max(distances)
#     d_['mean'] = np.mean(distances)
#     d_['std'] = np.std(distances)
#     d_['median'] = np.median(distances)
#     d_['kurtosis'] = kurtosis(distances)
#     d_['skew'] = skew(distances)
    
#     distances_bin = distances.astype(int)
#     for dis_bin in range(12):
#         d_[f'dis_bin_count{dis_bin}'] = np.where(distances_bin==dis_bin)[0].shape[0]
        
    list_.append(d_)
    idx += 1
    print(f'{idx}/{N_}',end='', flush=True)
    print('\r',end='',flush=True)
#     break

778648/1530476

In [103]:
123

123

In [104]:
df_list = pd.DataFrame(list_)

In [105]:
df_list.head()

,molecule_name,atom_index,w_C_max,w_C_min,w_C_mean,w_C_std,w_C_kurtosis,w_C_skew,w_H_max,w_H_min,...,w_O_mean,w_O_std,w_O_kurtosis,w_O_skew,w_F_max,w_F_min,w_F_mean,w_F_std,w_F_kurtosis,w_F_skew
0,dsgdb9nsd_000001,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.091953,1.091946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,dsgdb9nsd_000001,1,1.091953,1.091953,1.091953,0.0,-3.0,0.0,1.783157,1.783120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,dsgdb9nsd_000001,2,1.091952,1.091952,1.091952,0.0,-3.0,0.0,1.783158,1.783120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,dsgdb9nsd_000001,3,1.091946,1.091946,1.091946,0.0,-3.0,0.0,1.783158,1.783147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,dsgdb9nsd_000001,4,1.091948,1.091948,1.091948,0.0,-3.0,0.0,1.783157,1.783148,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
def _map_atom_charges(df, structures, atom_idx):
    df = pd.merge(df, structures, how = 'left', left_on  = ['molecule_name', f'atom_index_{atom_idx}'], right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)
    rename_list = {}
    for col in structures.columns.drop(['molecule_name',  'atom_index']).tolist():
        rename_list[col] = f'atom_index_{atom_idx}_{col}'
    df = df.rename(columns=rename_list)
    return df

In [107]:
train = pd.read_csv(f'{csv_file_folder}/train.csv')
train = train[train['type']=='1JHC']

In [116]:
train = _map_atom_charges(train, df_list, 0)
train = _map_atom_charges(train, df_list, 1)

In [117]:
LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'mae',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.2,
    'num_leaves': 256,
    'min_child_samples': 79,
    'max_depth': 9,
    'subsample_freq': 1,
    'subsample': 0.9,
    'bagging_seed': 11,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0
}

In [118]:
useless = ['id',
 'molecule_name',
 'atom_index_0',
 'atom_index_1',
 'type',
 'scalar_coupling_constant']

In [119]:
train.shape

(709416, 66)

In [120]:
columns = train.columns.drop(useless).tolist()

In [125]:
X_data = np.hstack((train[columns].values.astype('float32'), df_train2[columns0].values.astype('float32')))
y_data = train['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)

In [ ]:
model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
        verbose=100, early_stopping_rounds=200)

y_pred = model.predict(X_val)
np.log(mean_absolute_error(y_val, y_pred))

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's l1: 0.85273	valid_1's l1: 0.9382
[200]	valid_0's l1: 0.698458	valid_1's l1: 0.822076


In [ ]:
plt.figure(figsize=(10, 10))
cols = df_train2.columns.drop(['index', 'scalar_coupling_constant'])
df_importance = pd.DataFrame({'feature': columns + columns0, 'importance': model.feature_importances_})
sns.barplot(x="importance", y="feature", data=df_importance.sort_values('importance', ascending=False).head(20));